In [1]:
%load_ext autoreload
%pylab inline

import sys
import pandas as pd
import xarray as xr
import seaborn
from cartopy import crs, feature
from warnings import filterwarnings

seaborn.set_palette('tab20')
filterwarnings('ignore', category=RuntimeWarning)
%autoreload 2

sys.path.insert(0, '..')
# import the scripts package as reccap2ocean : r2o
import scripts as r2o

Populating the interactive namespace from numpy and matplotlib


# Import data using YAML file

Note that this should work on any machine as long as you have the requirements installed. Files will be downloaded to the 

In [2]:
cat = r2o.data.read_catalog('../reccap2_data.yml')

### SOCAT and SOCOM float (LIARv2)

In [4]:
# # read SOCAT data from the website
socat = r2o.data.socat2020(cat['socat_pco2'], verbose=False)
liar2 = r2o.data.soccom_float_liar(cat['soccom_liar'], verbose=False)

..............................................................................................................................................................................................

### RECCAP2 OBSERVATIONS: surface pCO2 data products

In [3]:
# read reccap data from the FTP site
# the output of reccap_surface_CO2 is a special object that I wrote some code for
# it contains all the netCDF files in the surface CO2 folder split by variable and then data provider
# this can be browsed interactively to see what is in the folder
# There are some faulty files on the server. I ran the script, updated the cache file and then ran it again. 
# there is a "specs" argument for the function below that gets the names of the models and variables. 
surface_co2 = r2o.data.obs_surface_co2(cat['surface_co2'], verbose=False, use_cache=True)

# MANUAL HACK
# CSIR and NIES have their fluxes -ve downward, change to +ve
transformations = {
    ('fgco2', 'NIES'): lambda a: -a,
    ('fgco2', 'CSIR'): lambda a: -a,
    ('fgco2_glob', 'NIES'): lambda a: -a,
    ('fgco2_glob', 'CSIR'): lambda a: -a,
    ('fgco2_reg',  'CSIR'): lambda a: -a,
}

for key, func in transformations.items():
    surface_co2[key[0]][key[1]] = func(surface_co2[key[0]][key[1]])
    
# the object can be used to quickly check the data entries for each variable. 
# note that some of the variables have been renamed based on input arguements.
# the unmatched file names are files that do not meet the matching requirements
# I write this to a file instead of showing in the summary here
with open('../data/raw/obs_surface_co2_summary.txt', 'w') as file:
    file.write(surface_co2.__str__())

In [8]:
model_surface_co2 = r2o.data.model_surface_co2(cat['model_surface_co2'], verbose=True, use_cache=True)

# MANUAL HACK
# based on looking at the files
transformations = {
    ('fgco2', 'CNRM_A'): lambda a: (a * 1000 / 12.011).assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'CNRM_B'): lambda a: (a * 1000 / 12.011).assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'GEOMAR_A'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'GEOMAR_B'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'GEOMAR_C'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'GEOMAR_D'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'NorESM_A'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'NorESM_B'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'NorESM_C'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'NorESM_D'): lambda a: a.assign_attrs(units='mol/m2/sec'),
    ('fgco2', 'BSOSE'): lambda a: -a
}
# for key, func in transformations.items():
#     model_surface_co2[key[0]][key[1]] = func(model_surface_co2[key[0]][key[1]])
    
# print(model_surface_co2.fgco2)

2021-04-13 08:27:39 [DOWNLOAD]  ================================================================================


2021-04-13 08:27:39 [DOWNLOAD]  Start of logging session
2021-04-13 08:27:39 [DOWNLOAD]  --------------------------------------------------------------------------------
2021-04-13 08:27:39 [DOWNLOAD]  Fetched 9 files from flist cache: ../data/raw/model_surface_co2/info_filelist.cache
2021-04-13 08:27:39 [DOWNLOAD]                    9 files ftp://ftp.bgc-jena.mpg.de/pub/outgoing/RECCAP2/32yTa68UrnNxJ6VYndOI/Ocean/Models/2D_CO2/*
2021-04-13 08:27:39 [DOWNLOAD]  9 files exist for ftp://ftp.bgc-jena.mpg.de/pub/outgoing/RECCAP2/32yTa68UrnNxJ6VYndOI/Ocean/Models/2D_CO2/*
2021-04-13 08:27:39 [DOWNLOAD]  Files will be saved to ../data/raw/model_surface_co2/
2021-04-13 08:27:39 [DOWNLOAD]  retrieving ftp://ftp.bgc-jena.mpg.de/pub/outgoing/RECCAP2/32yTa68UrnNxJ6VYndOI/Ocean/Models/2D_CO2/BSOSE_2D_CO2_I134.tar
2021-04-13 08:27:39 [DOWNLOAD]  retrieving ftp://ftp.bgc-jena.mpg.de/pub

In [9]:
model_surface_co2

<class 'scripts.data.main._RECCAP_dict'>
  Kw -------------------------------------------------------------------------------------------------- 
    BSOSE      [UNITS] mol C/m2/s       [DIMS] lon(360).lat(180).time(72)       [PREPROCESSING] renamed [t=time x=lon y=lat]; time centered on 15th; transposed (lat lon time);
    ETHZ_A     [UNITS] m/s              [DIMS] lon(360).lat(180).time_mon(468)  [PREPROCESSING] transposed (lat lon time_mon);
    ETHZ_B     [UNITS] m/s              [DIMS] lon(360).lat(180).time_mon(468)  [PREPROCESSING] transposed (lat lon time_mon);
    ETHZ_C     [UNITS] m/s              [DIMS] lon(360).lat(180).time_mon(468)  [PREPROCESSING] transposed (lat lon time_mon);
    ETHZ_D     [UNITS] m/s              [DIMS] lon(360).lat(180).time_mon(468)  [PREPROCESSING] transposed (lat lon time_mon);
    FESOM_A    [UNITS] -                [DIMS] lon(360).lat(180).time(468)      [PREPROCESSING] renamed [Lon=lon Lat=lat Time=time]; time centered on 15th; transposed (la

In [ ]:
fgco2 = model_surface_co2.fgco2.data() * 86400 * 365 * 12.011
names = [s for s in fgco2['variable'].values if re.findall('_A|BSO', s)]
modelA_fgco2 = fgco2.sel(variable=names).where(so, drop=True)

In [11]:
with open('../data/raw/model_surface_co2_summary.txt', 'w') as file:
    file.write(model_surface_co2.__str__())